## Imports

In [ ]:
import pandas as pd
import numpy as np
import scipy
from scipy import sparse
from scipy.sparse import csr_matrix, lil_matrix
import pickle
import math

## Preparing Matrices

In [ ]:
# the number of features
n = 61188
#number of classes
K = 20
#training instances
M = 12000

training_labels = np.load('../res/lr_labels.npy')
weights_matrix = np.random.rand(K, n + 1)
# weights_sparse is the W matrix in the pdf
weights_sparse = sparse.csr_matrix(weights_matrix)
training_data_sparse = sparse.load_npz('../res/lr_training_data.npz')
print(weights_sparse.shape)
print(training_data_sparse.shape)

XT = training_data_sparse.transpose()
print(scipy.sparse.issparse(XT))
print(XT.shape)

delta = lil_matrix(np.zeros((K, M)), dtype = np.int16)
for i,label in enumerate(training_labels):
    delta[label - 1, i] = 1
delta = delta.tocsr()
print(delta.shape)
# you see the index 1 mapping here: we need to be careful!
#how to getb the true class of a training examples
print(delta[:,0].nonzero()[0][0] + 1)
print(training_labels[0])
print(sparse.issparse(delta))

In [ ]:
ETHA = 0.005
LAMBDA = 0.01
XT = sparse.load_npz('../res/lr_training_data.npz').transpose()
for i in range(50):
    #print(weights_sparse)    
    pxy = (weights_sparse * XT).todense()
    for j in range(M):
        pxy[:,j] /= np.sum(pxy[:,j])
    pxy = np.exp(pxy)
    for j in range(M):
        pxy[:,j] /= np.sum(pxy[:,j])
    #print(pxy)
    diff_exp = csr_matrix(delta - pxy)
    new_w = weights_sparse + ETHA * (diff_exp * training_data_sparse - weights_sparse.multiply(LAMBDA))
    weights_sparse = new_w
    #print('new')
    #np.nan_to_num(weights_sparse, copy=False)



Test Prints

In [ ]:
print(weights_sparse.shape)
print(weights_sparse)


In [ ]:
correct = 0
# xxx importing the normalized xs here BAD PROGRAMMING PRACTICE FIX WHEN NOT BLEEDING!
predictions = (weights_sparse * XT).todense()
for j in range(M):
    predictions[:,j] /= np.sum(predictions[:,j])
predictions = np.exp(predictions)
for j in range(M):
    predictions[:,j] /= np.sum(predictions[:,j])
for i in range(predictions.shape[1]):
    #print(predictions[:,i])
    predicted = np.argmax(predictions[:,i]) + 1
    actural = training_labels[i]
    if predicted == actural:
        correct += 1
print(f"Corect is {correct} and accuracy is {correct/predictions.shape[1]}")
